# Trabalho Final

## Utilizando agrupamento nebuloso para analisar relação das vulnerabilidades demográficas e socioeconômicas dos municípios brasileiros com os efeitos da Covid-19

### Preparação dos dados 

In [1]:
import pandas as pd
from sklearn import preprocessing
import skfuzzy.cluster as cluster
import numpy as np
import matplotlib.pyplot as plt
import os


#### Dados epidemiológicos

In [95]:
directory_covid = os.fsencode('dados/covid')
df_covid = None
for file in os.listdir(directory_covid):
    filename = 'dados/covid/' + os.fsdecode(file)
    df_temp = pd.read_csv(filename, decimal=',', sep=';')
    if df_covid is None:
        df_covid = df_temp
    else:
        df_covid = pd.concat([df_covid, df_temp])

df_covid = df_covid[df_covid.municipio.isnull() == False]
df_covid.data = pd.to_datetime(df_covid.data, infer_datetime_format=True)
data_maxima = df_covid.data.max()
df_covid = df_covid[df_covid.data == data_maxima]

df_covid = df_covid.rename(
    columns={'regiao': 'REGIAO', 'estado': 'SIGLA_ESTADO', 'municipio': 'MUNICIPIO', 'coduf': 'CODIGO_ESTADO',
             'codmun': 'CODIGO_MUNICIPIO_6', 'codRegiaoSaude': 'CODIGO_REGIAO_SAUDE', 'nomeRegiaoSaude': 'REGIAO_SAUDE',
             'casosAcumulado': 'CASOS', 'obitosAcumulado': 'OBITOS', 'interior/metropolitana': 'REGIAO_METROPOLITANA'})

df_covid.CODIGO_MUNICIPIO_6 = df_covid.CODIGO_MUNICIPIO_6.astype('Int64').astype(str)

df_base = df_covid[['MUNICIPIO', 'CODIGO_MUNICIPIO_6', 'SIGLA_ESTADO', 'CODIGO_ESTADO', 'REGIAO', 'REGIAO_SAUDE',
                    'CODIGO_REGIAO_SAUDE', 'REGIAO_METROPOLITANA', 'CASOS', 'OBITOS']]

#### Nome dos estados

In [96]:
df_estados = pd.read_csv('dados/estados.csv', usecols=[1, 2])
df_estados.columns = ['ESTADO', 'SIGLA_ESTADO']
df_estados.SIGLA_ESTADO = df_estados.SIGLA_ESTADO.str.strip()
df_base = pd.merge(df_base, df_estados, how='left', on='SIGLA_ESTADO')

#### Aglomerados subnormais - Estimativa 2019

In [97]:
df_aglomerados_subnormais_estimativa = pd.read_excel('dados/IBGE/aglomerados_subnormais_estiamativa_2019.xls',
                                                     usecols=[1, 3, 4, 5, 8])
df_aglomerados_subnormais_estimativa.columns = \
    ['CODIGO_MUNICIPIO_7', 'POPULACAO_ESTIMADA', 'DOMICILIOS_PARTICULARES_OCUPADOS_ESTIMADO',
     'DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS_ESTIMADO',
     'PERCENTUAL_DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS_ESTIMADO']
df_aglomerados_subnormais_estimativa.CODIGO_MUNICIPIO_7 = df_aglomerados_subnormais_estimativa.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)
df_aglomerados_subnormais_estimativa['CODIGO_MUNICIPIO_6'] = df_aglomerados_subnormais_estimativa.CODIGO_MUNICIPIO_7.str[0:6]
df_aglomerados_subnormais_estimativa = df_aglomerados_subnormais_estimativa.fillna(0)

df_base = pd.merge(df_base, df_aglomerados_subnormais_estimativa, how='left', on=['CODIGO_MUNICIPIO_6'])

#### Regiões imediatas e regiões intermediárias

In [98]:
df_regioes = pd.read_excel('dados/IBGE/regioes_geograficas_composicao_por_municipios_2017_20180911.ods',
                           usecols=[1, 2, 3, 4, 5])
df_regioes.columns = ['CODIGO_MUNICIPIO_7', 'CODIGO_REGIAO_IMEDIATA', 'REGIAO_IMEDIATA', 'CODIGO_REGIAO_INTERMEDIARIA', 'REGIAO_INTERMEDIARIA']
df_regioes.CODIGO_MUNICIPIO_7 = df_regioes.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)
df_base = pd.merge(df_base, df_regioes, how='left', on=['CODIGO_MUNICIPIO_7'])

#### Censo 2010 - Dados IBGE

##### Faixa etária

In [99]:
df_percentual_faixa_etaria = pd.read_csv('dados/IBGE/tab2_percentual_faixa_etaria.csv', sep=';', decimal=',')
df_percentual_faixa_etaria = df_percentual_faixa_etaria.dropna()
df_percentual_faixa_etaria.columns = df_percentual_faixa_etaria.columns.str.replace(" ", "_")
df_percentual_faixa_etaria.columns = df_percentual_faixa_etaria.columns.str.upper()
df_percentual_faixa_etaria.CODMUN7 = df_percentual_faixa_etaria.CODMUN7.astype('Int64').astype(str)
df_percentual_faixa_etaria = df_percentual_faixa_etaria[
    ['CODMUN7', '0_A_5_ANOS', '6_A_14_ANOS', '15_A_24_ANOS', '25_A_39_ANOS', '40_A_59_ANOS',
     '60_ANOS_OU_MAIS']]
df_percentual_faixa_etaria.columns = \
    ['CODIGO_MUNICIPIO_7', 'POPULACAO_0_A_5_ANOS', 'POPULACAO_6_A_14_ANOS', 'POPULACAO_15_A_24_ANOS',
     'POPULACAO_25_A_39_ANOS', 'POPULACAO_40_A_59_ANOS', 'POPULACAO_60_ANOS_OU_MAIS']
df_base = pd.merge(df_base, df_percentual_faixa_etaria, how="left", on=['CODIGO_MUNICIPIO_7'])

##### População sede municipal, área, densidade e domicílios

In [100]:
PATH_SINOPSE = 'dados/IBGE/SinopseEstados/'
directory_sinopse = os.fsencode(PATH_SINOPSE)
df_1_densidade = None
df_2_domicilios = None
for directory_state in os.listdir(directory_sinopse):
    directory_path = PATH_SINOPSE + '/' + os.fsdecode(directory_state)
    for file in os.listdir(directory_path):
        filename = os.fsdecode(file)

        if filename.startswith('tab2_1_'):
            relative_filename = directory_path + '/' + filename
            df_temp = pd.read_excel(relative_filename, header=None, usecols=[3, 7, 8, 9],
                                 skiprows=10, skipfooter=1)
            df_temp.columns = ['POPULACAO_SEDE_MUNICIPAL', 'AREA', 'DENSIDADE', 'CODIGO_MUNICIPIO_7']
            if df_1_densidade is None:
                df_1_densidade = df_temp
            else:
                df_1_densidade = pd.concat([df_1_densidade, df_temp])

        if filename.startswith('tab2_2_'):
            relative_filename = directory_path + '/' + filename
            df_temp = pd.read_excel(relative_filename, header=None, usecols=[2, 11],
                                 skiprows=10, skipfooter=2)
            df_temp.columns = ['DOMICILIOS_PARTICULARES_OCUPADOS', 'CODIGO_MUNICIPIO_7']
            if df_2_domicilios is None:
                df_2_domicilios = df_temp
            else:
                df_2_domicilios = pd.concat([df_2_domicilios, df_temp])
df_1_densidade.CODIGO_MUNICIPIO_7 = df_1_densidade.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)
df_2_domicilios.CODIGO_MUNICIPIO_7 = df_2_domicilios.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)

df_base = pd.merge(df_base, df_1_densidade, how = 'left', on=['CODIGO_MUNICIPIO_7'])
df_base = pd.merge(df_base, df_2_domicilios, how = 'left', on=['CODIGO_MUNICIPIO_7'])

##### Aglomerados subnormais - Censo 2010

In [101]:
df_aglomerados_subnormais = pd.read_excel('dados/IBGE/aglomerados_subnormais.xls', header=None,
                                          usecols=[1, 2, 3, 4, 5, 6, 7], skiprows=8, skipfooter=3)
df_aglomerados_subnormais.columns = \
    ['AGLOMERADOS', 'DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS',
     'POPULACAO_AGLOMERADOS', 'AREA_AGLOMERADOS', 'DENSIDADE_DEMOGRAFICA_AGLOMERADOS',
     'DENSIDADE_DOMICILIOS_AGLOMERADOS','CODIGO_MUNICIPIO_7']
df_aglomerados_subnormais.CODIGO_MUNICIPIO_7 = df_aglomerados_subnormais.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)
df_aglomerados_subnormais = df_aglomerados_subnormais[df_aglomerados_subnormais.CODIGO_MUNICIPIO_7.str.len() == 7]

df_base = pd.merge(df_base, df_aglomerados_subnormais, how='left', on=['CODIGO_MUNICIPIO_7'])

df_base.AGLOMERADOS = df_base.AGLOMERADOS.fillna(0)
df_base.DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS = df_base.DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS.fillna(0)
df_base.POPULACAO_AGLOMERADOS = df_base.POPULACAO_AGLOMERADOS.fillna(0)
df_base.AREA_AGLOMERADOS = df_base.AREA_AGLOMERADOS.fillna(0)
df_base.DENSIDADE_DEMOGRAFICA_AGLOMERADOS = df_base.DENSIDADE_DEMOGRAFICA_AGLOMERADOS.fillna(0)
df_base.DENSIDADE_DOMICILIOS_AGLOMERADOS = df_base.DENSIDADE_DOMICILIOS_AGLOMERADOS.fillna(0)

##### Atributos sistematizados pelo Atlas BR

In [102]:
df_censo = pd.read_csv('dados/AtlasBrasil/censo.csv', sep=';', decimal=',')
df_censo = df_censo[df_censo.ANO ==  2010]
df_censo.columns = df_censo.columns.str.upper()
df_censo = df_censo[['CODMUN7', 'IDHM','IDHM_E','IDHM_L','IDHM_R', 'FECTOT','CORTE1','CORTE2',
                    'CORTE3','CORTE4','CORTE9','GINI', 'PIND', 'PMPOB', 'PPOB', 'PREN10RICOS', 'PREN20','PREN20RICOS',
                    'PREN40','PREN60','PREN80', 'PRENTRAB','R1040','R2040', 'RDPC1','RDPC10','RDPC2','RDPC3','RDPC4',
                    'RDPC5','RDPCT','RIND','RMPOB','RPOB','THEIL', 'CPR','EMP','P_AGRO','P_COM','P_CONSTR','P_EXTR',
                    'P_FORMAL', 'P_SERV', 'P_SIUP', 'P_TRANSF', 'REN0','REN1','REN2','REN3','REN5', 'T_ATIV', 'T_DES',
                    'THEILTRAB', 'TRABPUB','T_BANAGUA', 'T_DENS', 'T_OCUPDESLOC_1', 'HOMEMTOT',
                     'MULHERTOT', 'PESO1', 'PESO13', 'PESO4', 'PESO5', 'PESO610',
                    'PESO1114', 'PESO1517', 'PESO18', 'PESO65', 'PESORUR','PESOTOT','PESOURB', 'RAZDEP', 'POP']]
df_censo = df_censo.rename(
    columns={'CODMUN7': 'CODIGO_MUNICIPIO_7', 'IDHM_E': 'IDHM_EDUCACAO','IDHM_L': 'IDHM_LONGEVIDADE','IDHM_R': 'IDHM_RENDA',
             'FECTOT': 'TAXA_FECUNDIDADE','CORTE1': 'RENDA_PER_CAPTA_MAXIMA_1_QUINTO_MAIS_POBRE',
             'CORTE2': 'RENDA_PER_CAPTA_MAXIMA_2_QUINTO_MAIS_POBRE', 'CORTE3': 'RENDA_PER_CAPTA_MAXIMA_3_QUINTO_MAIS_POBRE',
             'CORTE4': 'RENDA_PER_CAPTA_MAXIMA_4_QUINTO_MAIS_POBRE','CORTE9': 'RENDA_PER_CAPTA_MINIMA_10_PORCENTO_MAIS_RICOS',
             'PIND': 'PERCENTUAL_EXTREMAMENTE_POBRES', 'PMPOB': 'PERCENTUAL_POBRES', 'PPOB': 'PERCENTUAL_VULNERAVEIS_POBREZA',
             'PREN10RICOS': 'PERCENTUAL_RENDA_APROPRIADA_10_PORCENTO_MAIS_RICOS',
             'PREN20RICOS': 'PERCENTUAL_RENDA_APROPRIADA_20_PORCENTO_MAIS_RICOS',
             'PREN20': 'PERCENTUAL_RENDA_APROPRIADA_20_PORCENTO_MAIS_POBRES',
             'PREN40': 'PERCENTUAL_RENDA_APROPRIADA_40_PORCENTO_MAIS_POBRES',
             'PREN60': 'PERCENTUAL_RENDA_APROPRIADA_60_PORCENTO_MAIS_POBRES',
             'PREN80': 'PERCENTUAL_RENDA_APROPRIADA_80_PORCENTO_MAIS_POBRES',
             'PRENTRAB': 'PERCENTUAL_RENDA_PROVENIENTE_TRABALHO','R1040': 'RAZAO_10_PORCENTO_MAIS_RICOS_POR_40_PORCENTO_MAIS_POBRES',
             'R2040': 'RAZAO_20_PORCENTO_MAIS_RICOS_POR_40_PORCENTO_MAIS_POBRES',
             'RDPC1': 'RENDA_PER_CAPTA_MEDIA_1_QUINTO_MAIS_POBRE', 'RDPC2': 'RENDA_PER_CAPTA_MEDIA_2_QUINTO_MAIS_POBRE',
             'RDPC3': 'RENDA_PER_CAPTA_MEDIA_3_QUINTO_MAIS_POBRE','RDPC4': 'RENDA_PER_CAPTA_MEDIA_4_QUINTO_MAIS_POBRE',
             'RDPC5': 'RENDA_PER_CAPTA_MEDIA_20_PORCENTO_MAIS_RICOS', 'RDPC10': 'RENDA_PER_CAPTA_MEDIA_10_PORCENTO_MAIS_RICOS',
             'RIND': 'RENDA_PER_CAPTA_MEDIA_EXTREMAMENTE_POBRES','RMPOB': 'RENDA_PER_CAPTA_MEDIA_POBRES',
             'RPOB': 'RENDA_PER_CAPTA_MEDIA_VULNERAVEIS_POBREZA','CPR': 'PERCENTUAL_TRABALHADORES_CONTA_PROPRIA',
             'P_AGRO': 'PERCENTUAL_OCUPADOS_AGROPECUARIA','P_COM': 'PERCENTUAL_OCUPADOS_COMERCIO','P_CONSTR': 'PERCENTUAL_OCUPADOS_CONSTRUCAO',
             'P_EXTR': 'PERCENTUAL_OCUPADOS_MINERACAO', 'P_FORMAL': 'PERCENTUAL_TRABALHADORES_FORMALIZADOS',
             'P_SERV': 'PERCENTUAL_OCUPADOS_SERVICO', 'P_SIUP': 'PERCENTUAL_OCUPADOS_SERVICOS_INDUSTRIAIS_UTILIDADE_PUBLICA',
             'P_TRANSF': 'PERCENTUAL_OCUPADOS_INDUSTRIA', 'REN0': 'PERCENTUAL_OCUPADOS_SEM_RENDA',
             'REN1': 'PERCENTUAL_OCUPADOS_ATE_1_SALARIO_MINIMO','REN2': 'PERCENTUAL_OCUPADOS_ATE_2_SALARIOS_MINIMOS',
             'REN3': 'PERCENTUAL_OCUPADOS_ATE_3_SALARIOS_MINIMOS','REN5': 'PERCENTUAL_OCUPADOS_ATE_5_SALARIOS_MINIMOS',
             'T_ATIV': 'TAXA_ATIVIDADE', 'T_DES': 'TAXA_DESOCUPACAO', 'THEILTRAB': 'THEIL_TRABALHO', 'TRABPUB': 'PERCENTUAL_TRABALHADORES_PUBLICOS',
             'T_BANAGUA': 'PERCENTUAL_POPULACAO_VIVE_DOMICILIOS_BANHEIRO_AGUA',
             'T_DENS': 'PERCENTUAL_POPULACAO_VIVE_DOMICILIOS_DENSIDADE_MAIOR_2_POR_DORMITORIO',
             'T_OCUPDESLOC_1': 'PERCENTUAL_POPULACAO_VULNERAVEL_POBREZA_GASTAM_MAIS_UMA_HORA_ATE_TRABALHO',
             'HOMEMTOT': 'POPULACAO_MASCULINA', 'MULHERTOT': 'POPULACAO_FEMININA', 'PESO1': 'POPULACAO_ATE_1_ANO',
             'PESO13': 'POPULACAO_1_A_3_ANOS', 'PESO4': 'POPULACAO_4_ANOS', 'PESO5': 'POPULACAO_5_ANOS',
             'PESO610': 'POPULACAO_6_A_10_ANOS', 'PESO1114': 'POPULACAO_11_A_14_ANOS', 'PESO1517': 'POPULACAO_15_A_17_ANOS',
             'PESO18': 'POPULACAO_18_ANOS_OU_MAIS', 'PESO65': 'POPULACAO_65_ANOS_OU_MAIS', 'PESORUR': 'POPULACAO_RURAL',
             'PESOTOT': 'POPULACAO_TOTAL','PESOURB': 'POPULACAO_URBANA', 'RAZDEP': 'RAZAO_DEPENDENCIA', 'POP': 'POPULACAO_DOMICILIOS_PARTICULARES'})
df_censo = df_censo.astype({'CODIGO_MUNICIPIO_7': str})

df_base = pd.merge(df_base, df_censo, how='left', on='CODIGO_MUNICIPIO_7')

#### Registros administrativos sistematizados pelo Atlas BR

In [103]:
df_registros_administrativos =  pd.read_csv('dados/AtlasBrasil/registros_administrativos.csv', decimal=',', sep=';')
df_registros_administrativos = df_registros_administrativos.astype({'IBGE7': str})

df_registros_administrativos_2017 = df_registros_administrativos[df_registros_administrativos.ANO == 2017]
df_registros_administrativos_2017 = df_registros_administrativos_2017[
    ['IBGE7','PINTERDRSAI', 'REN_TRPCBF_D', 'TXCOBPLP']]
df_registros_administrativos_2017 = df_registros_administrativos_2017.rename(
    columns={'IBGE7': 'CODIGO_MUNICIPIO_7', 'PINTERDRSAI': 'PERCENTUAL_INTERNACAOES_DOENCAS_SANEAMENTO_INADEQUADO',
             'REN_TRPCBF_D': 'TRANSFERENCIA_PERCAPTA_BOLSA_FAMILIA', 'TXCOBPLP': 'PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE'})
df_base = pd.merge(df_base, df_registros_administrativos_2017, how="left", on=['CODIGO_MUNICIPIO_7'])

df_registros_administrativos_2016 = df_registros_administrativos[df_registros_administrativos.ANO == 2016]
df_registros_administrativos_2016 = df_registros_administrativos_2016[
    ['IBGE7', 'REN_PVAADM', 'REN_PVAAGR', 'REN_PVAIND', 'REN_PVASERV']]
df_registros_administrativos_2016 = df_registros_administrativos_2016.rename(
    columns={'IBGE7': 'CODIGO_MUNICIPIO_7', 'REN_PVAADM': 'PERCENTUAL_VALOR_ADICIONADO_ADMINISTRACAO_PUBLICA',
             'REN_PVAAGR': 'PERCENTUAL_VALOR_ADICIONADO_AGRICULTURA', 'REN_PVAIND': 'PERCENTUAL_VALOR_ADICIONADO_INDUSTRIA',
             'REN_PVASERV': 'PERCENTUAL_VALOR_ADICIONADO_SERVICO'})
df_base = pd.merge(df_base, df_registros_administrativos_2016, how="left", on=['CODIGO_MUNICIPIO_7'])


##### Tratamento de dados ausentes em PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE

In [104]:
df_group_regiao_imediata = df_base.groupby('CODIGO_REGIAO_IMEDIATA').mean()[['PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE']]
df_group_regiao_imediata.columns = ['PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE_MEDIA_REGIAO_IMEDIATA']
df_group_regiao_imediata = df_group_regiao_imediata.reset_index()
df_base = pd.merge(df_base, df_group_regiao_imediata, how='left', on='CODIGO_REGIAO_IMEDIATA')
df_base.loc[df_base.PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE.isnull(), 'PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE'] = \
    df_base[df_base.PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE.isnull()].PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE_MEDIA_REGIAO_IMEDIATA
df_base = df_base.drop(columns=['PERCENTUAL_POPULACAO_COBERTA_PLANO_SAUDE_MEDIA_REGIAO_IMEDIATA'])

#### Eleições 2018

In [105]:
PATH_ELEICOES = 'dados/Eleicao2018/'
directory_eleicoes = os.fsencode(PATH_ELEICOES)
df_eleicoes = None
for directory_state in os.listdir(directory_eleicoes):
    directory_path = PATH_ELEICOES + '/' + os.fsdecode(directory_state)
    for file in os.listdir(directory_path):
        filename = os.fsdecode(file)

        if filename.endswith('.csv'):
            relative_filename = directory_path + '/' + filename
            df_eleicoes_temp = pd.read_csv(relative_filename, encoding='Latin 1', sep=';')
            df_eleicoes_temp = df_eleicoes_temp[df_eleicoes_temp.DS_CARGO_PERGUNTA == 'Presidente']
            df_eleicoes_temp = df_eleicoes_temp.groupby(['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NM_VOTAVEL']).sum()
            df_eleicoes_temp = df_eleicoes_temp[['QT_VOTOS']]
            df_eleicoes_branco = df_eleicoes_temp.loc[(slice(None), slice(None), 'Branco')]
            df_eleicoes_branco.columns = ['VOTOS_BRANCO']
            df_eleicoes_nulo = df_eleicoes_temp.loc[(slice(None), slice(None), 'Nulo')]
            df_eleicoes_nulo.columns = ['VOTOS_NULO']
            df_eleicoes_bolsonaro = df_eleicoes_temp.loc[(slice(None), slice(None), 'JAIR BOLSONARO')]
            df_eleicoes_bolsonaro.columns = ['VOTOS_BOLSONARO']
            df_eleicoes_haddad = df_eleicoes_temp.loc[(slice(None), slice(None), 'FERNANDO HADDAD')]
            df_eleicoes_haddad.columns = ['VOTOS_HADDAD']
            df_eleicoes_temp = pd.merge(df_eleicoes_bolsonaro, df_eleicoes_haddad, left_index=True, right_index=True)
            df_eleicoes_temp = pd.merge(df_eleicoes_temp, df_eleicoes_branco, left_index=True, right_index=True)
            df_eleicoes_temp = pd.merge(df_eleicoes_temp, df_eleicoes_nulo, left_index=True, right_index=True)
            df_eleicoes_temp = df_eleicoes_temp.reset_index(level=['CD_MUNICIPIO', 'NM_MUNICIPIO'])
            if df_eleicoes is None:
                df_eleicoes = df_eleicoes_temp
            else:
                df_eleicoes = pd.concat([df_eleicoes, df_eleicoes_temp])

df_eleicoes = df_eleicoes.drop(columns=['NM_MUNICIPIO'])
df_eleicoes = df_eleicoes.rename(columns={'CD_MUNICIPIO': 'CODIGO_MUNICIPIO_TSE'})
df_codigos_tse_ibge = pd.read_csv('dados/municipios_brasileiros_tse.csv', usecols=[0, 3, 4])
df_codigos_tse_ibge.columns = ['CODIGO_MUNICIPIO_TSE', 'CAPITAL', 'CODIGO_MUNICIPIO_7']
df_eleicoes = pd.merge(df_eleicoes, df_codigos_tse_ibge, how="left", on=['CODIGO_MUNICIPIO_TSE'])
df_eleicoes.CODIGO_MUNICIPIO_7 = df_eleicoes.CODIGO_MUNICIPIO_7.astype('Int64').astype(str)

df_base = pd.merge(df_base, df_eleicoes, how='left', on=['CODIGO_MUNICIPIO_7'])

/home/helder/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/helder/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (25,27,30,32,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/helder/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30,32,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [108]:
df_base.head()

,MUNICIPIO,CODIGO_MUNICIPIO_6,SIGLA_ESTADO,CODIGO_ESTADO,REGIAO,REGIAO_SAUDE,CODIGO_REGIAO_SAUDE,REGIAO_METROPOLITANA,CASOS,OBITOS,...,PERCENTUAL_VALOR_ADICIONADO_ADMINISTRACAO_PUBLICA,PERCENTUAL_VALOR_ADICIONADO_AGRICULTURA,PERCENTUAL_VALOR_ADICIONADO_INDUSTRIA,PERCENTUAL_VALOR_ADICIONADO_SERVICO,CODIGO_MUNICIPIO_TSE,VOTOS_BOLSONARO,VOTOS_HADDAD,VOTOS_BRANCO,VOTOS_NULO,CAPITAL
0,Alta Floresta D'Oeste,110001,RO,11,Norte,ZONA DA MATA,11005.0,0.0,4040.0,65,...,31.40,36.55,6.88,25.18,310,8524,3763,169,626,0
1,Ariquemes,110002,RO,11,Norte,VALE DO JAMARI,11001.0,0.0,22424.0,493,...,29.94,7.37,17.95,44.73,78,37325,8862,754,2041,0
2,Cabixi,110003,RO,11,Norte,CONE SUL,11006.0,0.0,886.0,21,...,31.32,46.22,3.62,18.85,450,2142,1110,56,195,0
3,Cacoal,110004,RO,11,Norte,CAFE,11002.0,0.0,14386.0,302,...,27.73,10.74,13.15,48.38,94,35962,10091,848,2024,0
4,Cerejeiras,110005,RO,11,Norte,CONE SUL,11006.0,0.0,2514.0,61,...,28.44,15.12,6.41,50.03,272,7656,1745,121,412,0


#### Dados da saúde

##### Cobertura da Estratégia de Saúde da Família

In [140]:
TODO: PAREI AQUI

df_cobertura_estrategia_saude_familia = pd.read_excel('dados/saude/Cobertura-AB-TODOS OS MUNICÍPIOS-Dezembro de 2019.xls',
                                                      header=None, usecols=[3, 12], skiprows=9, skipfooter=6)
df_cobertura_estrategia_saude_familia.columns = \
    ['CODMUN6', 'PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA']
df_cobertura_estrategia_saude_familia.CODMUN6 = df_cobertura_estrategia_saude_familia.CODMUN6.astype('Int64').astype(str)
df_cobertura_estrategia_saude_familia.PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA = \
    df_cobertura_estrategia_saude_familia.PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA.str.replace("%","")
df_cobertura_estrategia_saude_familia.PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA = \
    df_cobertura_estrategia_saude_familia.PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA.str.replace(",",".")
df_cobertura_estrategia_saude_familia = df_cobertura_estrategia_saude_familia.astype({'PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA': float})

##### Cobertura dos Agentes Comunitários de Saúde

In [141]:
df_cobertura_agentes_saude = pd.read_excel('dados/saude/Cobertura-ACS-TODOS OS MUNICÍPIOS-Dezembro de 2019.xls',
                                                      header=None, usecols=[3, 8], skiprows=9, skipfooter=6)
df_cobertura_agentes_saude.columns = \
    ['CODMUN6', 'PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE']
df_cobertura_agentes_saude.CODMUN6 = df_cobertura_agentes_saude.CODMUN6.astype('Int64').astype(str)
df_cobertura_agentes_saude.PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE = \
    df_cobertura_agentes_saude.PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE.str.replace("%","")
df_cobertura_agentes_saude.PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE = \
    df_cobertura_agentes_saude.PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE.str.replace(",",".")
df_cobertura_agentes_saude = df_cobertura_agentes_saude.astype({'PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE': float})

##### Enfermeiros

In [142]:
df_enfermeiros = pd.read_csv('dados/saude/enfermeiros_2019_12.csv', sep=';', decimal=',')
df_enfermeiros.columns = ['MUNICIPIO', 'NUMERO_ENFERMEIROS_SUS', 'NUMERO_ENFERMEIROS_NAO_SUS', 'NUMERO_ENFERMEIROS_TOTAL']
df_enfermeiros =  df_enfermeiros.replace("-", 0)
df_enfermeiros['CODMUN6'] = df_enfermeiros.MUNICIPIO.str[:6]
df_enfermeiros = df_enfermeiros.astype({'NUMERO_ENFERMEIROS_SUS': int, 'NUMERO_ENFERMEIROS_NAO_SUS': int, 'NUMERO_ENFERMEIROS_TOTAL': int})
df_enfermeiros = df_enfermeiros.drop(columns=['MUNICIPIO'])

##### Médicos

In [143]:
df_medicos = pd.read_csv('dados/saude/medicos_2019_12.csv', sep=';', decimal=',')
df_medicos.columns = ['MUNICIPIO', 'NUMERO_MEDICOS_SUS', 'NUMERO_MEDICOS_NAO_SUS', 'NUMERO_MEDICOS_TOTAL']
df_medicos =  df_medicos.replace("-", 0)
df_medicos['CODMUN6'] = df_medicos.MUNICIPIO.str[:6]
df_medicos = df_medicos.astype({'NUMERO_MEDICOS_SUS': int, 'NUMERO_MEDICOS_NAO_SUS': int, 'NUMERO_MEDICOS_TOTAL': int})
df_medicos = df_medicos.drop(columns=['MUNICIPIO'])

##### LEITOS INTERNAÇÃO

In [144]:
df_leitos_internacao = pd.read_csv('dados/saude/leitos_internacao_2019_12.csv', sep=';', decimal=',')
df_leitos_internacao.columns = ['MUNICIPIO', 'NUMERO_LEITOS_INTERNACAO_SUS', 'NUMERO_LEITOS_INTERNACAO_NAO_SUS', 'NUMERO_LEITOS_INTERNACAO_TOTAL']
df_leitos_internacao =  df_leitos_internacao.replace("-", 0)
df_leitos_internacao['CODMUN6'] = df_leitos_internacao.MUNICIPIO.str[:6]
df_leitos_internacao = df_leitos_internacao.astype({'NUMERO_LEITOS_INTERNACAO_SUS': int, 'NUMERO_LEITOS_INTERNACAO_NAO_SUS': int, 'NUMERO_LEITOS_INTERNACAO_TOTAL': int})
df_leitos_internacao = df_leitos_internacao.drop(columns=['MUNICIPIO'])

##### LEITOS UTI

In [145]:
df_leitos_uti = pd.read_csv('dados/saude/leitos_uti_2019_12.csv', sep=';', decimal=',')
df_leitos_uti.columns = ['MUNICIPIO', 'NUMERO_LEITOS_UTI_SUS', 'NUMERO_LEITOS_UTI_NAO_SUS', 'NUMERO_LEITOS_UTI_TOTAL']
df_leitos_uti =  df_leitos_uti.replace("-", 0)
df_leitos_uti['CODMUN6'] = df_leitos_uti.MUNICIPIO.str[:6]
df_leitos_uti = df_leitos_uti.astype({'NUMERO_LEITOS_UTI_SUS': int, 'NUMERO_LEITOS_UTI_NAO_SUS': int, 'NUMERO_LEITOS_UTI_TOTAL': int})
df_leitos_uti = df_leitos_uti.drop(columns=['MUNICIPIO'])

##### Ventiladores/Respiradores

In [147]:
df_ventiladores_respiradores = pd.read_csv('dados/saude/respiradores_ventiladores_2019_12.csv',
                                           sep=';', decimal=',', usecols=[0, 1, 2])
df_ventiladores_respiradores.columns = ['MUNICIPIO', 'NUMERO_VENTILADORES_RESPIRADORES_EXISTENTES', 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO']
df_ventiladores_respiradores =  df_ventiladores_respiradores.replace("-", 0)
df_ventiladores_respiradores['CODMUN6'] = df_ventiladores_respiradores.MUNICIPIO.str[:6]
df_ventiladores_respiradores = df_ventiladores_respiradores.astype({'NUMERO_VENTILADORES_RESPIRADORES_EXISTENTES': int, 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO': int})
df_ventiladores_respiradores = df_ventiladores_respiradores.drop(columns=['MUNICIPIO'])

#### Doenças de risco à Covid-19

In [193]:
df_asma_estados = pd.read_excel('dados/pesquisa_nacional_saude/asma.ods', header=None, usecols=[0, 1], skiprows=14, skipfooter=2)
df_asma_estados.columns = ['ESTADO', 'PERCENTUAL_ASMA']
df_asma_estados.head()

,ESTADO,PERCENTUAL_ASMA
0,Rondônia,3.6
1,Acre,3.7
2,Amazonas,4.5
3,Roraima,6.1
4,Pará,4.6


In [195]:
df_asma_capitais = pd.read_excel('dados/pesquisa_nacional_saude/asma.ods', sheet_name='sexo_(cap)', header=None, usecols=[0, 1], skiprows=12, skipfooter=3)
df_asma_capitais.columns = ['MUNICIPIO', 'PERCENTUAL_ASMA']
df_asma_capitais.head()


,MUNICIPIO,PERCENTUAL_ASMA
0,Porto Velho,3.8
1,Rio Branco,4.0
2,Manaus,4.1
3,Boa Vista,6.6
4,Belém,6.1


##### Merge df_base and df_cobertura_estrategia_saude_familia

In [157]:
df_base = pd.merge(df_base, df_cobertura_estrategia_saude_familia, how='left', on=['CODMUN6'])


##### Merge df_base and df_cobertura_agentes_saude

In [158]:
df_base = pd.merge(df_base, df_cobertura_agentes_saude, how='left', on=['CODMUN6'])

##### Merge df_base and df_enfermeiros

In [159]:
df_base = pd.merge(df_base, df_enfermeiros, how='left', on=['CODMUN6'])

##### Merge df_base and df_medicos

In [160]:
df_base = pd.merge(df_base, df_medicos, how='left', on=['CODMUN6'])

##### Merge df_base and df_leitos_internacao

In [161]:
df_base = pd.merge(df_base, df_leitos_internacao, how='left', on=['CODMUN6'])

##### Merge df_base and df_leitos_uti

In [162]:
df_base = pd.merge(df_base, df_leitos_uti, how='left', on=['CODMUN6'])

##### Merge df_base and df_ventiladores_respiradores

In [163]:
df_base = pd.merge(df_base, df_ventiladores_respiradores, how='left', on=['CODMUN6'])

##### Merge df_base and df_group_regiao_imediata

In [ ]:
df_group_regiao_imediata = df_base.groupby('CODIGO_REGIAO_IMEDIATA').sum()[
    ['POPULACAO_ESTIMADA', 'NUMERO_LEITOS_INTERNACAO_TOTAL', 'NUMERO_LEITOS_UTI_TOTAL', 'NUMERO_MEDICOS_TOTAL',
     'NUMERO_ENFERMEIROS_TOTAL', 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO']]
df_group_regiao_imediata.columns = ['POPULACAO_ESTIMADA_REGIAO_IMEDIATA', 'NUMERO_LEITOS_INTERNACAO_TOTAL_REGIAO_IMEDIATA',
                                    'NUMERO_LEITOS_UTI_TOTAL_REGIAO_IMEDIATA', 'NUMERO_MEDICOS_TOTAL_REGIAO_IMEDIATA',
     'NUMERO_ENFERMEIROS_TOTAL_REGIAO_IMEDIATA', 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO_REGIAO_IMEDIATA']
df_group_regiao_imediata = df_group_regiao_imediata.reset_index()
df_base = pd.merge(df_base, df_group_regiao_imediata, how='left', on='CODIGO_REGIAO_IMEDIATA')

##### Merge df_base and df_group_regiao_intermediaria

In [ ]:
df_group_regiao_intermediaria = df_base.groupby('CODIGO_REGIAO_INTERMEDIARIA').sum()[
    ['POPULACAO_ESTIMADA', 'NUMERO_LEITOS_INTERNACAO_TOTAL', 'NUMERO_LEITOS_UTI_TOTAL', 'NUMERO_MEDICOS_TOTAL',
     'NUMERO_ENFERMEIROS_TOTAL', 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO']]
df_group_regiao_intermediaria.columns = ['POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA', 'NUMERO_LEITOS_INTERNACAO_TOTAL_REGIAO_INTERMEDIARIA',
                                    'NUMERO_LEITOS_UTI_TOTAL_REGIAO_INTERMEDIARIA', 'NUMERO_MEDICOS_TOTAL_REGIAO_INTERMEDIARIA',
     'NUMERO_ENFERMEIROS_TOTAL_REGIAO_INTERMEDIARIA', 'NUMERO_VENTILADORES_RESPIRADORES_EM_USO_REGIAO_INTERMEDIARIA']
df_group_regiao_intermediaria = df_group_regiao_intermediaria.reset_index()
df_base = pd.merge(df_base, df_group_regiao_intermediaria, how='left', on='CODIGO_REGIAO_INTERMEDIARIA')

#### Creating new attributes

In [165]:
df_base['TAXA_CASOS_100_MIL'] = df_base.CASOS / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_OBITOS_100_MIL'] = df_base.OBITOS / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_LETALIDADE'] = df_base.OBITOS / df_base.CASOS * 100
df_base['RAZAO_HOMEM_MULHER'] = df_base.HOMEMTOT / df_base.MULHERTOT
df_base['RAZAO_POPULACAO_RURAL_URBANA'] = df_base.PESORUR / df_base.PESOURB
df_base['PERCENTUAL_POPULACAO_SEDE_MUNICIPAL'] = df_base.POPULACAO_SEDE_MUNICIPAL / df_base.PESOTOT * 100
df_base['PERCENTUAL_POPULACAO_17_OU_MENOS_ANOS'] = \
    (df_base.PESO1 + df_base.PESO13 + df_base.PESO4 + df_base.PESO5 + df_base.PESO610 + df_base.PESO1114 + df_base.PESO1517) / df_base.PESOTOT * 100
df_base['PERCENTUAL_POPULACAO_18_ATE_39_ANOS'] = (df_base.PESO18 / df_base.PESOTOT * 100) - df_base['40_A_59_ANOS'] - df_base['60_ANOS_OU_MAIS']
df_base['PERCENTUAL_POPULACAO_40_ATE_59_ANOS'] = df_base['40_A_59_ANOS']
df_base['PERCENTUAL_POPULACAO_60_OU_MAIS_ANOS'] = df_base['60_ANOS_OU_MAIS']
df_base['DENSIDADE_DOMICILIOS'] = df_base.TOTAL_DOMICILIOS_OCUPADOS / df_base.AREA
df_base['PERCENTUAL_DOMICILIOS_AGLOMERADOS'] = df_base.NUMERO_DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS / df_base.TOTAL_DOMICILIOS_OCUPADOS * 100
df_base['PERCENTUAL_POPULACAO_AGLOMERADOS'] = df_base.POPULACAO_DOMICILIOS_PARTICULARES_OCUPADOS_AGLOMERADOS / df_base.PESOTOT * 100
df_base['DENSIDADE_DOMICILIOS_ESTIMADA'] = df_base.EDOC_TOTAL / df_base.AREA
df_base['NUMERO_VOTOS'] = df_base.VOTOS_BOLSONARO + df_base.VOTOS_HADDAD + df_base.VOTOS_NULO + df_base.VOTOS_BRANCO
df_base['PERCENTUAL_VOTOS_BOLSONARO'] = df_base.VOTOS_BOLSONARO / df_base.NUMERO_VOTOS * 100
df_base['PERCENTUAL_VOTOS_HADDAD'] = df_base.VOTOS_HADDAD / df_base.NUMERO_VOTOS * 100
df_base['TAXA_LEITOS_INTERNACAO_100_MIL'] = df_base.NUMERO_LEITOS_INTERNACAO_TOTAL / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_LEITOS_UTI_100_MIL'] = df_base.NUMERO_LEITOS_UTI_TOTAL / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_MEDICOS_100_MIL'] = df_base.NUMERO_MEDICOS_TOTAL / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_ENFERMEIROS_100_MIL'] = df_base.NUMERO_ENFERMEIROS_TOTAL / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_VENTILADORES_REPIRADORES_EM_USO_100_MIL'] = df_base.NUMERO_VENTILADORES_RESPIRADORES_EM_USO / df_base.POPULACAO_ESTIMADA * 100000
df_base['TAXA_LEITOS_INTERNACAO_REGIAO_IMEDIATA_100_MIL'] = df_base.NUMERO_LEITOS_INTERNACAO_TOTAL_REGIAO_IMEDIATA / df_base.POPULACAO_ESTIMADA_REGIAO_IMEDIATA * 100000
df_base['TAXA_LEITOS_UTI_REGIAO_IMEDIATA_100_MIL'] = df_base.NUMERO_LEITOS_UTI_TOTAL_REGIAO_IMEDIATA / df_base.POPULACAO_ESTIMADA_REGIAO_IMEDIATA * 100000
df_base['TAXA_MEDICOS_REGIAO_IMEDIATA_100_MIL'] = df_base.NUMERO_MEDICOS_TOTAL_REGIAO_IMEDIATA / df_base.POPULACAO_ESTIMADA_REGIAO_IMEDIATA * 100000
df_base['TAXA_ENFERMEIROS_REGIAO_IMEDIATA_100_MIL'] = df_base.NUMERO_ENFERMEIROS_TOTAL_REGIAO_IMEDIATA / df_base.POPULACAO_ESTIMADA_REGIAO_IMEDIATA * 100000
df_base['TAXA_VENTILADORES_REPIRADORES_EM_USO_REGIAO_IMEDIATA_100_MIL'] = df_base.NUMERO_VENTILADORES_RESPIRADORES_EM_USO_REGIAO_IMEDIATA / df_base.POPULACAO_ESTIMADA_REGIAO_IMEDIATA * 100000
df_base['TAXA_LEITOS_INTERNACAO_REGIAO_INTERMEDIARIA_100_MIL'] = df_base.NUMERO_LEITOS_INTERNACAO_TOTAL_REGIAO_INTERMEDIARIA / df_base.POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA * 100000
df_base['TAXA_LEITOS_UTI_REGIAO_INTERMEDIARIA_100_MIL'] = df_base.NUMERO_LEITOS_UTI_TOTAL_REGIAO_INTERMEDIARIA / df_base.POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA * 100000
df_base['TAXA_MEDICOS_REGIAO_INTERMEDIARIA_100_MIL'] = df_base.NUMERO_MEDICOS_TOTAL_REGIAO_INTERMEDIARIA / df_base.POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA * 100000
df_base['TAXA_ENFERMEIROS_REGIAO_INTERMEDIARIA_100_MIL'] = df_base.NUMERO_ENFERMEIROS_TOTAL_REGIAO_INTERMEDIARIA / df_base.POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA * 100000
df_base['TAXA_VENTILADORES_REPIRADORES_EM_USO_REGIAO_INTERMEDIARIA_100_MIL'] = df_base.NUMERO_VENTILADORES_RESPIRADORES_EM_USO_REGIAO_INTERMEDIARIA / df_base.POPULACAO_ESTIMADA_REGIAO_INTERMEDIARIA * 100000

#### Analizing the attributes

In [ ]:
df_base.head()

In [ ]:
df_base.describe()

In [ ]:
df_base.IDHM_R.hist(legend=True)
df_base.IDHM_E.hist(legend=True)
df_base.IDHM_L.hist(legend=True)

In [ ]:
df_base.TAXA_CASOS_100_MIL.hist(legend=True)

In [ ]:
df_base.TAXA_OBITOS_100_MIL.hist(legend=True)

In [ ]:
df_base.POPULACAO_ESTIMADA.hist(legend=True)

In [ ]:
df_base.TAXA_LETALIDADE.hist(legend=True)
df_base.plot.scatter('IDHM_E', 'IDHM_R')
df_base.plot.scatter('IDHM_R', 'IDHM_L')
df_base.plot.scatter('IDHM_E', 'IDHM_L')
df_base.plot.scatter('IDHM_R', 'TAXA_CASOS_100_MIL')
df_base.plot.scatter('IDHM_R', 'TAXA_OBITOS_100_MIL')
df_base.plot.scatter('IDHM_R', 'TAXA_LETALIDADE')
df_base.plot.scatter('TAXA_OBITOS_100_MIL', 'TAXA_CASOS_100_MIL')
df_base.plot.scatter('TAXA_LETALIDADE', 'TAXA_OBITOS_100_MIL')
df_base.plot.scatter('TAXA_CASOS_100_MIL', 'TAXA_LETALIDADE')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'IDHM_E')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'IDHM_R')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'IDHM_L')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'TAXA_OBITOS_100_MIL')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'TAXA_CASOS_100_MIL')
df_base.plot.scatter('POPULACAO_ESTIMADA', 'TAXA_LETALIDADE')

In [ ]:
df_base.boxplot(column=["IDHM_R", "IDHM_E", "IDHM_L"])

In [ ]:
df_base.boxplot(column=["POPULACAO_ESTIMADA"])

In [ ]:
df_base.boxplot(column="TAXA_LETALIDADE")

In [ ]:
df_base.boxplot(column=["TAXA_OBITOS_100_MIL"])

In [ ]:
df_base.boxplot(column=["TAXA_CASOS_100_MIL"])

#### Removing outliers

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out


#### Drop missing values, normalizing data, and selecting attributes

In [ ]:
df_base = df_base.dropna()

df_antecedents_not_normalized = df_base[['POPULACAO_ESTIMADA', 'IDHM', 'IDHM_E', 'IDHM_L',
                                         'IDHM_R', 'PINTERDRSAI', 'REN_RENSF_D', 'REN_TRPCBF_D', 'REN_PVAADM',
                                         'REN_PVAAGR', 'REN_PVAIND', 'REN_PVASERV', 'TXCOBPLP', 'REGIAO_METROPOLITANA', 'FECTOT',
                                         'CORTE1','CORTE2', 'CORTE3','CORTE4','CORTE9','GINI', 'PIND', 'PMPOB', 'PPOB',
                                         'PREN10RICOS', 'PREN20','PREN20RICOS', 'PREN40','PREN60','PREN80', 'PRENTRAB',
                                         'R1040','R2040', 'RDPC1','RDPC10','RDPC2','RDPC3','RDPC4', 'RDPC5','RDPCT',
                                         'RIND','RMPOB','RPOB','THEIL', 'CPR','EMP','P_AGRO','P_COM','P_CONSTR','P_EXTR',
                    'P_FORMAL', 'P_SERV', 'P_TRANSF', 'REN0','REN1','REN2','REN3','REN5', 'T_ATIV', 'T_DES',
                    'THEILTRAB', 'TRABCC','TRABPUB','TRABSC', 'T_AGUA','T_BANAGUA', 'T_DENS', 'T_LUZ','AGUA_ESGOTO',
                    'T_OCUPDESLOC_1', 'PEA', 'RAZAO_HOMEM_MULHER',
                                         'RAZAO_POPULACAO_RURAL_URBANA', 'PERCENTUAL_POPULACAO_17_OU_MENOS_ANOS',
                                         'PERCENTUAL_POPULACAO_18_ATE_39_ANOS', 'PERCENTUAL_POPULACAO_40_ATE_59_ANOS',
                                         'PERCENTUAL_POPULACAO_60_OU_MAIS_ANOS', 'RAZDEP',
                                         'PERCENTUAL_POPULACAO_SEDE_MUNICIPAL', 'DENSIDADE', 'DENSIDADE_DOMICILIOS',
                                         'PERCENTUAL_DOMICILIOS_AGLOMERADOS', 'PERCENTUAL_POPULACAO_AGLOMERADOS',
                                         'DENSIDADE_DOMICILIOS_ESTIMADA', 'PER_AGSN', 'PERCENTUAL_VOTOS_BOLSONARO',
                                         'PERCENTUAL_VOTOS_HADDAD', 'CAPITAL',
                                         'PERCENTUAL_COBERTURA_ESTRATEGIA_SAUDE_FAMILIA',
                                         'PERCENTUAL_COBERTURA_AGENTES_COMUNITARIOS_SAUDE',
                                         'TAXA_LEITOS_INTERNACAO_100_MIL',
                                         'TAXA_LEITOS_UTI_100_MIL',
                                         'TAXA_MEDICOS_100_MIL',
                                         'TAXA_ENFERMEIROS_100_MIL', 'TAXA_VENTILADORES_REPIRADORES_EM_USO_100_MIL']]
scaler_antecedents = preprocessing.MinMaxScaler()
antecedents_normalized = scaler_antecedents.fit_transform(df_antecedents_not_normalized.values)
df_antecedents = pd.DataFrame(antecedents_normalized, columns=df_antecedents_not_normalized.columns)

df_consequents_not_normalized = df_base[['TAXA_CASOS_100_MIL', 'TAXA_OBITOS_100_MIL', 'TAXA_LETALIDADE']]
scaler_concequents = preprocessing.MinMaxScaler()
concequents_normalized = scaler_concequents.fit_transform(df_consequents_not_normalized.values)
df_concequents = pd.DataFrame(concequents_normalized, columns=df_consequents_not_normalized.columns)

In [ ]:
df_antecedents.IDHM_R.hist(legend=True)
df_antecedents.IDHM_E.hist(legend=True)
df_antecedents.IDHM_L.hist(legend=True)
df_antecedents.POPULACAO_ESTIMADA.hist(legend=True)

In [ ]:
df_concequents.TAXA_CASOS_100_MIL.hist(legend=True)

In [ ]:
df_concequents.TAXA_OBITOS_100_MIL.hist(legend=True)

In [ ]:
df_concequents.TAXA_LETALIDADE.hist(legend=True)

#### Executing fcmenas

In [ ]:
from random import randrange


def plot_clusters(centers, u, x_not_normalized, y_not_normalized, df_not_normalized, scaler):
    cluster_membership = np.argmax(u, axis=0)  # Hardening for visualization
    fig, ax = plt.subplots()

    for j in range(len(centers)):
        ax.plot(x_not_normalized.values[cluster_membership == j],
                y_not_normalized.values[cluster_membership == j], '.', label="Cluster " + str(j))
        ax.set_xlabel(x_not_normalized.name)
        ax.set_ylabel(y_not_normalized.name)
    ax.legend()

    centers_not_normalized = scaler.inverse_transform(centers)
    xi = df_not_normalized.columns.get_loc(x_not_normalized.name)
    yi = df_not_normalized.columns.get_loc(y_not_normalized.name)
    for pt in centers_not_normalized:
        ax.plot(pt[xi], pt[yi], 'rs')

    plt.show()


def describe_clusters(centers, u, df_not_normalized):
    cluster_membership = np.argmax(u, axis=0)
    for j in range(len(centers)):
        print("===================Cluster {0}==================".format(j))
        for column  in df_not_normalized.columns:
            print(df_not_normalized[cluster_membership == j][column].describe())
            print("\n")


def show_results(ra, centers, u, fpc, df_not_normalized, scaler, df_analysis):
    if ra is not None:
        print('RA = {0} - Subtractive Centers = {1}; FPC = {2:.2f}'.format(ra, len(centers), fpc))
    else:
        print('Centers = {0}; FPC = {1:.2f}'.format(len(centers), fpc))
    attributes = df_not_normalized.columns

    # for i in range(len(attributes) - 1):
    #     for j in range(i + 1, len(attributes)):
    #         plot_clusters(centers, u, df_not_normalized[attributes[i]], df_not_normalized[attributes[j]],
    #                       df_not_normalized, scaler)
    i = randrange(len(attributes))
    j = randrange(len(attributes))
    plot_clusters(centers, u, df_not_normalized[attributes[i]], df_not_normalized[attributes[j]], df_not_normalized,
                  scaler)

    print("Describing clusters")
    describe_clusters(centers, u, df_not_normalized)

    print("Describing results")
    describe_clusters(centers, u, df_analysis)

In [ ]:
m = 2
epsilon = 10 ** (-4)


In [ ]:
# ra_25=0.25
# centers_25, u_25, u0, d, jm, p, fpc_25 = cluster.cmeans(
#     df_antecedents.values.T, c=None, m=m, error=epsilon, maxiter=1000, init=None,
#     subtractive_clustering_influence_range=ra_25)

In [ ]:
# centers_25_c, u_25_c, u0, d, jm, p, fpc_25_c = cluster.cmeans(
#     df_concequents.values.T, c=None, m=m, error=epsilon, maxiter=1000, init=None,
#     subtractive_clustering_influence_range=ra_25)

In [ ]:
# show_results(ra_25, centers_25, u_25, fpc_25, df_antecedents_not_normalized, scaler_antecedents)
# show_results(ra_25, centers_25_c, u_25_c, fpc_25_c, df_consequents_not_normalized, scaler_concequents)

In [ ]:
# ra_19=0.19
# centers_19, u_19, u0, d, jm, p, fpc_19 = cluster.cmeans(
#     df_antecedents.values.T, c=None, m=m, error=epsilon, maxiter=1000, init=None,
#     subtractive_clustering_influence_range=ra_19)

In [ ]:
# centers_19_c, u_19_c, u0, d, jm, p, fpc_19_c = cluster.cmeans(
#     df_concequents.values.T, c=None, m=m, error=epsilon, maxiter=1000, init=None,
#     subtractive_clustering_influence_range=ra_19)

In [ ]:
# show_results(ra_19, centers_19, u_19, fpc_19, df_antecedents_not_normalized, scaler_antecedents)
# show_results(ra_19, centers_19_c, u_19_c, fpc_19_c, df_consequents_not_normalized, scaler_concequents)
#

In [ ]:
centers_2, u_2, u0, d, jm, p, fpc_2 = cluster.cmeans(
    df_antecedents.values.T, c=2, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
centers_2_c, u_2_c, u0, d, jm, p, fpc_2_c = cluster.cmeans(
    df_concequents.values.T, c=2, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
show_results(None, centers_2, u_2, fpc_2, df_antecedents_not_normalized, scaler_antecedents, df_consequents_not_normalized)
show_results(None, centers_2_c, u_2_c, fpc_2_c, df_consequents_not_normalized, scaler_concequents, df_antecedents_not_normalized)

In [ ]:
centers_3, u_3, u0, d, jm, p, fpc_3 = cluster.cmeans(
    df_antecedents.values.T, c=3, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
centers_3_c, u_3_c, u0, d, jm, p, fpc_3_c = cluster.cmeans(
    df_concequents.values.T, c=3, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
show_results(None, centers_3, u_3, fpc_3, df_antecedents_not_normalized, scaler_antecedents, df_consequents_not_normalized)
show_results(None, centers_3_c, u_3_c, fpc_3_c, df_consequents_not_normalized, scaler_concequents, df_antecedents_not_normalized)

In [ ]:
centers_4, u_4, u0, d, jm, p, fpc_4 = cluster.cmeans(
    df_antecedents.values.T, c=4, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
centers_4_c, u_4_c, u0, d, jm, p, fpc_4_c = cluster.cmeans(
    df_concequents.values.T, c=4, m=m, error=epsilon, maxiter=1000, init=None)

In [ ]:
show_results(None, centers_4, u_4, fpc_4, df_antecedents_not_normalized, scaler_antecedents, df_consequents_not_normalized)
show_results(None, centers_4_c, u_4_c, fpc_4_c, df_consequents_not_normalized, scaler_concequents, df_antecedents_not_normalized)